# 1. Imports

In [1]:
from sklearn.decomposition import PCA
import matplotlib as mpl 
import numpy as np
import os
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import f1_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler


from codes.utils import stratified_train_test_group_kfold
from codes.utils import model_test_classification

from codes.classification_codes import rnc_gridsearch



0.10.1


# 2. Data

In [2]:
with open("Data/MoS2_Analysis_Data_trained2", "rb") as fp:   # Unpickling
    MoS2_Proj1_Class_Data = pickle.load(fp)

#features = MoS2_ImageNet_100_data[0]
df = pd.DataFrame(MoS2_Proj1_Class_Data)
print(df.keys())
T_dict= {900:0, 950:1, 1000:2}
T_target = np.array([T_dict[item] for item in df['T']])

features = np.array(list(df['ImageNet']))
sampleId = np.array(list(df['sampleId']))

X = features
Y = np.array(T_target)
groups = np.array(sampleId)


train_val_groups, train_val_X, train_val_Y, test_X, test_Y = stratified_train_test_group_kfold(X, Y, groups, n_splits=10, test_fold=0)

train_val_Y = train_val_Y.flatten()
test_Y = test_Y.flatten()

print(train_val_X.shape)
print(train_val_Y.shape)
print(test_X.shape)
print(test_Y.shape)


oversample = RandomOverSampler(sampling_strategy='not majority')

scaler = StandardScaler().fit(train_val_X)
train_val_X = scaler.transform(train_val_X)
test_X = scaler.transform(test_X)

Index(['sampleId', 'sampleLabel', 'image', 'T', 'ImageNet', 'MicroNet'], dtype='object')


# 3. Runing Cross-Validation

In [5]:
Radius = [i for i in range(2, 7)]
Weights = ['uniform', 'distance']
Algorithm = ['auto', 'ball_tree', 'kd_tree', 'brute']
P = [i for i in range(1, 6)]


def cross_10_folds_rnn(train_val_X, train_val_Y):
    
    best_train = []
    best_val = []
    best_variables = []
    for fold in range(10):
        group, train_X, train_Y, val_X, val_Y = stratified_train_test_group_kfold(train_val_X, train_val_Y, train_val_groups, n_splits=10, test_fold=fold)
        
        model_path = f'classification/ImageNet/aug3/RNC_{fold}_model.sav'
        
        train_X, train_Y = oversample.fit_resample(train_X, train_Y)
        variables_best, performance_best = rnn_gridsearch(train_X, train_Y, val_X, val_Y, Radius, Weights, Algorithm, P, fold, model_path)
        best_train.append(performance_best['accuracy_train'])
        best_val.append(performance_best['accuracy_val'])
        best_variables.append(variables_best)
        print(f'fold: {fold} done!')
    return best_train, best_val, best_variables


best_train, best_val, best_variables = cross_10_folds_rnn(train_val_X, train_val_Y)
print(best_train)
print(best_val)
print(f'train_acc_mean: {np.mean(best_train) :.3f}, std: {np.std(best_train) :.3f}, val_acc_mean: {np.mean(best_val):.3f}, std: {np.std(best_val):.3f}')

<class 'generator'>
best_train_acc: 1.0, best_val_acc: 0.125
best_train_acc: 0.9904761904761905, best_val_acc: 0.16666666666666666
best_train_acc: 0.8, best_val_acc: 0.25
best_train_acc: 0.7111111111111111, best_val_acc: 0.4583333333333333
best_train_acc: 0.6888888888888889, best_val_acc: 0.6666666666666666
best_train_acc: 0.546031746031746, best_val_acc: 0.7083333333333334
best_train_acc: 0.5428571428571428, best_val_acc: 0.75
best_train_acc: 1.0, best_val_acc: 0.7916666666666666
fold: 0 done!
<class 'generator'>
best_train_acc: 1.0, best_val_acc: 0.125
best_train_acc: 0.9591194968553459, best_val_acc: 0.16666666666666666
best_train_acc: 0.7767295597484277, best_val_acc: 0.375
best_train_acc: 0.6981132075471698, best_val_acc: 0.4583333333333333
best_train_acc: 1.0, best_val_acc: 0.5
best_train_acc: 0.6415094339622641, best_val_acc: 0.6666666666666666
best_train_acc: 1.0, best_val_acc: 0.75
best_train_acc: 1.0, best_val_acc: 0.7916666666666666
fold: 1 done!
<class 'generator'>
best_tra

# 4. Model Testing

In [6]:
best_fold = 0

model_path = f'classification/ImageNet/aug3/RNC_{best_fold}_model.sav'
# Fitting different train in the same hyperparameters
best_test, confusion_matrix_test = model_test_classification(train_val_X, train_val_Y, train_val_groups,test_X, test_Y, best_fold, model_path)



print(best_test)
print(f'test_acc_mean: {np.mean(best_test) :.3f}, std: {np.std(best_test) :.3f}')

print('5th confusion matrix: ', [list(item) for item in confusion_matrix_test[4]])

<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
[0.6296296296296297, 0.6666666666666666, 0.6296296296296297, 0.6666666666666666, 0.6296296296296297, 0.5925925925925926, 0.6666666666666666, 0.6296296296296297, 0.6666666666666666, 0.6666666666666666]
test_acc_mean: 0.644, std: 0.025
5th confusion matrix:  [[1, 2, 0], [1, 11, 1], [0, 6, 5]]


In [7]:
cm_list = []
for cm in confusion_matrix_test:
    cm_list.append([list(item) for item in cm])
#print(cm_list)

print(len(cm_list))
print([list(item) for item in np.mean(cm_list, axis=0)])

10
[[1.6, 1.4, 0.0], [1.2, 10.8, 1.0], [0.9, 5.1, 5.0]]


In [7]:
PATH = os.path.join('Models', model_path)
loaded_model = pickle.load(open(PATH, 'rb'))
loaded_model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'outlier_label': 0,
 'p': 5,
 'radius': 6,
 'weights': 'distance'}